# Artificial Neural Network

### Importing the libraries

In [93]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [94]:
print(tf.__version__)

2.14.0


## Part 1 - Data Preprocessing

### Importing the dataset

In [95]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [96]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

One Hot Encoding the "Geography" column

In [97]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough"
)
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [98]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [99]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [100]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [101]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the second hidden layer

In [102]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer

In [103]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3 - Training the ANN

### Compiling the ANN

In [104]:
ann.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

### Training the ANN on the Training set

In [105]:
ann.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
250/250 [==============================] - 0s 457us/step - loss: 0.5622 - accuracy: 0.7880
Epoch 2/10
250/250 [==============================] - 0s 432us/step - loss: 0.4821 - accuracy: 0.7960
Epoch 3/10
250/250 [==============================] - 0s 430us/step - loss: 0.4487 - accuracy: 0.7966
Epoch 4/10
250/250 [==============================] - 0s 424us/step - loss: 0.4262 - accuracy: 0.8061
Epoch 5/10
250/250 [==============================] - 0s 425us/step - loss: 0.4078 - accuracy: 0.8216
Epoch 6/10
250/250 [==============================] - 0s 423us/step - loss: 0.3924 - accuracy: 0.8303
Epoch 7/10
250/250 [==============================] - 0s 426us/step - loss: 0.3807 - accuracy: 0.8428
Epoch 8/10
250/250 [==============================] - 0s 426us/step - loss: 0.3715 - accuracy: 0.8486
Epoch 9/10
250/250 [==============================] - 0s 423us/step - loss: 0.3648 - accuracy: 0.8546
Epoch 10/10
250/250 [==============================] - 0s 423us/step - loss: 0.359

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [106]:
named_encoder = ct.named_transformers_['encoder']
print("Learned Categories (Countries):", named_encoder.categories_[0])

encoded_country = named_encoder.transform([['France']]).toarray()
print(f"\nEncoding for France:\n", encoded_country)

Learned Categories (Countries): ['France' 'Germany' 'Spain']

Encoding for France:
 [[1. 0. 0.]]


In [107]:
prob_leaving = ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(f"propability of this customer leaves the banke is {100 * prob_leaving}%")

1/1 [==============================] - 0s 34ms/step
propability of this customer leaves the banke is [[3.6515586]]%


### Predicting the Test set results

In [108]:
y_test_predict = ann.predict(X_test)
y_test_predict = (y_test_predict > 0.5)
print(
    np.concatenate(
        (
            y_test_predict.reshape(len(y_test_predict), 1),
            y_test.reshape(len(y_test), 1),
        ),
        axis=1,
    )
)

63/63 [==============================] - 0s 392us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [109]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_test_predict, normalize='all'))
print(accuracy_score(y_test, y_test_predict, normalize=True))

[[0.7685 0.029 ]
 [0.107  0.0955]]
0.864
